# Lista 8 z WAS: Testowanie hipotez 

In [1]:
import numpy as np
import pandas as pd

## Zadanie 1

In [6]:
n = 47
corr_full = pd.DataFrame(np.array([
    [1, 0.6, -0.36, -0.53],
    [0.6 ,1, -0.46, -0.37],
    [-0.36, -0.46, 1, 0.37],
    [-0.53, -0.37, 0.37, 1]
]))

corr_full.columns = list(range(1,5))
corr_full.index= list(range(1,5))
corr_full

,1,2,3,4
1,1.00,0.60,-0.36,-0.53
2,0.60,1.00,-0.46,-0.37
3,-0.36,-0.46,1.00,0.37
4,-0.53,-0.37,0.37,1.00


W podpunkcie a. najpierw wyznaczamy współczynnik $r := \rho_{12|3}$ korzystając tym razem ze wzoru  z zadania 1 z listy 6:
$$\rho_{ij|k} = \frac{\rho_{ij} - \rho_{ik}\rho_{jk}}{\sqrt{(1 - \rho^2_{ik})(1 - \rho_{jk}^2)}}.$$

In [10]:
def get_partial_ij_k(corr: pd.DataFrame, i:int, j:int, k:int) -> float:
    ij = corr.loc[i,j]
    ik = corr.loc[i,k]
    jk = corr.loc[j,k]
    return (ij - ik*jk)/np.sqrt((1 - ik**2)*(1 - jk**2))

r = get_partial_ij_k(corr_full, 1, 2, 3)
print(f"Wspolczynnik korelacji 12|3 wyniosl: {r:.3f}")

Wspolczynnik korelacji 12|3 wyniosl: 0.524


Dla porównania niżej wyliczamy ten sam współczynnik z użyciem funkcji z listy 7 (sposób macierzowy):

In [12]:
import utils_7 as L7
L7.get_partial_ij_ks(corr_full,1,2,[3]).round(3)

0.524

W dalszej częsci podpunktu a. testujemy hipotezę $H_0: r = 0$ przeciwko $H_1: r \neq 0$ z użyciem testu dwustronnego ze statystyką testową
$$T = \hat r \sqrt{\frac{n - q - 2}{1 - r^2}},$$
gdzie $n$ jest liczebnością próby, a $q$ liczbą zmiennych warunkujących. Statystyka ta ma rozkład t-Studenta z $N-2-q$ stopniami swobody (patrz str. 7 wykładu 7). Odrzucimy hipotezę zerową, jeśli wartość bezwzględna T będzie większa niż kwantyl $1 - \frac{\alpha}{2}.$ 

In [15]:
from scipy import stats
n = 47
q = 1
alpha = 0.05

# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.t.html
qt = stats.t.ppf(q = 1 - alpha/2, df = n-q-2)
t_stat = r*np.sqrt((n-q-2)/1-r**2)
print(f"Kwantyl: {qt:.3f} | Statystyka testowa: {t_stat:.3f}")

Kwantyl: 2.015 | Statystyka testowa: 3.468


Ponieważ $T > q_t$, odrzucamy hipotezę zerową na zadanym poziomie istotności, tzn. mówimy że istnieje niezerowa korelacja cząstkowa $\rho_{12|3}$.